# Requires TF2
https://github.com/eleozzr/desc/blob/master/desc_2.1.1_paul.ipynb

In [ ]:
!pip install desc

In [ ]:
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
from collections import Counter
import glob2
import tensorflow as tf
import time
import desc
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2
print(desc.__version__, tf.__version__)

In [ ]:
for category in [ "real_data", "balanced_data", "imbalanced_data"
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal", "features"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
    #         tf.reset_default_graph()
            save_dir="desc"
            adata = sc.AnnData(X)
            adata.obs['Group'] = Y
            sc.pp.log1p(adata)
            sc.pp.highly_variable_genes(adata,n_top_genes=1000,subset=True,inplace=True)

            sc.pp.scale(adata,max_value=6)
            adata=desc.train(adata,
                    dims=[adata.shape[1],64,32],
                    tol=0.005,
                    n_neighbors=10,
                    batch_size=256,
                    louvain_resolution=[1.0],# not necessarily a list, you can only set one value, like, louvain_resolution=1.0
                    save_dir=str(save_dir),
                    do_tsne=True,
                    learning_rate=200, # the parameter of tsne
                    use_GPU=True,
                    num_Cores=1, #for reproducible, only use 1 cpu
                    num_Cores_tsne=4,
                    save_encoder_weights=False,
                    save_encoder_step=3,# save_encoder_weights is False, this parameter is not used
                    use_ae_weights=False,
                    do_umap=False)
            elapsed = time.time() - start
            truth = adata.obs["Group"]
            pred = adata.obs["desc_1.0"]
            ARI = np.around(adjusted_rand_score(truth, pred), 5)
            NMI = np.around(normalized_mutual_info_score(truth, pred), 5)
            ss = silhouette_score(adata.obsm['X_Embeded_z1.0'], pred)
            cal = calinski_harabasz_score(adata.obsm['X_Embeded_z1.0'],pred)
            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run, elapsed, pred, cal, adata.obsm['X_Embeded_z1.0']]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_desc.pkl")
